In [3]:
import numpy as np
import tensorflow as tf

2022-05-10 22:44:30.587861: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-10 22:44:30.587895: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [4]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  2704k      0  0:00:30  0:00:30 --:--:-- 2263k      0  0:00:29  0:00:28  0:00:01 2696k


# Movie Review recommendation

In [5]:
!ls aclImdb

README	imdb.vocab  imdbEr.txt	test  train


In [6]:
!ls aclImdb/test

labeledBow.feat  neg  pos  urls_neg.txt  urls_pos.txt


In [7]:
!ls aclImdb/train

labeledBow.feat  pos	unsupBow.feat  urls_pos.txt
neg		 unsup	urls_neg.txt   urls_unsup.txt


In [8]:
!cat aclImdb/train/pos/6248_7.txt

Being an Austrian myself this has been a straight knock in my face. Fortunately I don't live nowhere near the place where this movie takes place but unfortunately it portrays everything that the rest of Austria hates about Viennese people (or people close to that region). And it is very easy to read that this is exactly the directors intention: to let your head sink into your hands and say "Oh my god, how can THAT be possible!". No, not with me, the (in my opinion) totally exaggerated uncensored swinger club scene is not necessary, I watch porn, sure, but in this context I was rather disgusted than put in the right context.<br /><br />This movie tells a story about how misled people who suffer from lack of education or bad company try to survive and live in a world of redundancy and boring horizons. A girl who is treated like a whore by her super-jealous boyfriend (and still keeps coming back), a female teacher who discovers her masochism by putting the life of her super-cruel "lover" 

In [9]:
!tree -d aclImdb

aclImdb
├── test
│   ├── neg
│   └── pos
└── train
    ├── neg
    ├── pos
    └── unsup

7 directories


In [10]:
!rm -r aclImdb/train/unsup # remove unused dir

In [11]:
batch_size = 32
raw_train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    './aclImdb/train/', batch_size=batch_size, validation_split=.2, 
    subset='training', seed=1337
)
raw_val_ds = tf.keras.preprocessing.text_dataset_from_directory(
    './aclImdb/train/', batch_size=batch_size, validation_split=.2, 
    subset='validation', seed=1337
)
raw_test_ds = tf.keras.preprocessing.text_dataset_from_directory(
    './aclImdb/train/', batch_size=batch_size
)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.


2022-05-10 22:46:46.437431: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-05-10 22:46:46.437471: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-05-10 22:46:46.437494: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (liberio-X550JX): /proc/driver/nvidia/version does not exist
2022-05-10 22:46:46.914913: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
Found 25000 files belonging to 2 classes.


In [12]:
print(f"Number of batches in raw_train_ds: {raw_train_ds.cardinality()}")
print(f"Number of batches in raw_val_ds: {raw_val_ds.cardinality()}")
print(f"Number of batches in raw_test_ds: {raw_test_ds.cardinality()}")

Number of batches in raw_train_ds: 625
Number of batches in raw_val_ds: 157
Number of batches in raw_test_ds: 782


In [13]:
for text_batch, label_batch in raw_train_ds.take(1):
    for i in range(3):
        print(text_batch.numpy()[i], end='\n\n\t')
        print(label_batch.numpy()[i], end='\n -----------------')

b'I\'ve seen tons of science fiction from the 70s; some horrendously bad, and others thought provoking and truly frightening. Soylent Green fits into the latter category. Yes, at times it\'s a little campy, and yes, the furniture is good for a giggle or two, but some of the film seems awfully prescient. Here we have a film, 9 years before Blade Runner, that dares to imagine the future as somthing dark, scary, and nihilistic. Both Charlton Heston and Edward G. Robinson fare far better in this than The Ten Commandments, and Robinson\'s assisted-suicide scene is creepily prescient of Kevorkian and his ilk. Some of the attitudes are dated (can you imagine a filmmaker getting away with the "women as furniture" concept in our oh-so-politically-correct-90s?), but it\'s rare to find a film from the Me Decade that actually can make you think. This is one I\'d love to see on the big screen, because even in a widescreen presentation, I don\'t think the overall scope of this film would receive its

# Prepare data 
### remove \</br>

In [14]:
import string
import re

In [15]:
def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, '<br />', '')
    return tf.strings.regex_replace(
        stripped_html, f"[{re.escape(string.punctuation)}]", ""
    )

In [16]:
max_features = 20000
embedding_dim = 128
sequence_length = 500

In [17]:
vectorize_layer = tf.keras.layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length
)

In [18]:
text_ds = raw_train_ds.map(lambda x,y : x)
vectorize_layer.adapt(text_ds)

In [19]:
text_input = tf.keras.Input(shape=(1,), dtype=tf.string, name='text')
x = vectorize_layer(text_input)
x = tf.keras.layers.Embedding(max_features+1, embedding_dim)(x)


# Make Model

In [ ]:
inputs = keras.Input(shape=(None,), dtype="Int64")

x = tf.keras.layers.Embedding(max_features, embedding_dim)(inputs)
x = tf.keras.layers.Dropoutopout(.5)(x)

from tensorflow.keras import layers
x = layers.Conv1D(128,)

In [22]:
import sys
sys.path # path of callable module (python module only)

['/home/liberio/Documents/Data Science/ML/Tensorflow/Keras/Introduction/Text classification',
 '/home/liberio/anaconda3/lib/python39.zip',
 '/home/liberio/anaconda3/lib/python3.9',
 '/home/liberio/anaconda3/lib/python3.9/lib-dynload',
 '',
 '/home/liberio/.local/lib/python3.9/site-packages',
 '/home/liberio/anaconda3/lib/python3.9/site-packages',
 '/home/liberio/anaconda3/lib/python3.9/site-packages/locket-0.2.1-py3.9.egg',
 '/home/liberio/anaconda3/lib/python3.9/site-packages/IPython/extensions',
 '/home/liberio/.ipython']

In [24]:
import math

SyntaxError: invalid syntax (4229062975.py, line 2)